<a href="https://colab.research.google.com/github/Gaurab-Kharal/Advance-data-analysis---sql/blob/main/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

In [20]:
df = pd.read_csv('laptops_data.csv')

In [21]:
df.sample(3)

,Unnamed: 0.1,Unnamed: 0,brand,name,price,spec_rating,processor,CPU,Ram,Ram_type,ROM,ROM_type,GPU,display_size,resolution_width,resolution_height,OS,warranty
578,603,683,Lenovo,IdeaPad Slim 3 82RK00VWIN Laptop,38390,69.323529,12th Gen Intel Core i3 1215U,"Hexa Core (2P + 4E), 8 Threads",8GB,DDR4,512GB,SSD,Intel UHD Graphics,15.6,1920.0,1080.0,Windows 11 OS,1
447,468,532,HP,Pavilion 14s- fy1003AU Laptop,45890,69.323529,5th Gen AMD Ryzen 5 5500U,"Hexa Core, 12 Threads",16GB,DDR4,512GB,SSD,AMD Radeon Graphics,14.0,1920.0,1080.0,Windows 11 OS,1
190,196,221,HP,Victus 15-fb0777AX Gaming Laptop,51990,67.000000,5th Gen AMD Ryzen 5 5600H,"Hexa Core, 12 Threads",8GB,DDR4,512GB,SSD,4GB AMD Radeon RX 6500M Graphics,15.6,1920.0,1080.0,Windows 11 OS,1


In [22]:
df["Ram"] = df["Ram"].str.replace("GB","").astype(int)

In [23]:
df = df.drop(columns=['Unnamed: 0.1','Unnamed: 0','name'])

In [24]:
import re
import numpy as np
import pandas as pd

# --- helper functions ---
def to_gb_storage(x):
    if pd.isna(x):
        return np.nan
    s = str(x).upper().replace(" ", "")
    # handle mixed like "512GBSSD" => keep numbers before GB/TB
    m = re.search(r'([\d\.]+)\s*TB', s)
    if m:
        return float(m.group(1)) * 1024
    m = re.search(r'([\d\.]+)\s*GB', s)
    if m:
        return float(m.group(1))
    # fallback: try to parse digits
    m = re.search(r'([\d\.]+)', s)
    return float(m.group(1)) if m else np.nan

def parse_ram(x):
    if pd.isna(x): return np.nan
    s = str(x).upper()
    m = re.search(r'(\d+)', s)
    return int(m.group(1)) if m else np.nan

def gpu_brand_and_vram(x):
    if pd.isna(x): return ("Other", np.nan)
    s = str(x)
    s_low = s.lower()
    # vram first
    m = re.search(r'(\d+)\s*GB', s, flags=re.IGNORECASE)
    vram = int(m.group(1)) if m else np.nan
    if "nvidia" in s_low:
        return ("NVIDIA", vram)
    if "geforce" in s_low:  # some strings omit 'nvidia' but include 'GeForce'
        return ("NVIDIA", vram)
    if "radeon" in s_low or "amd" in s_low:
        return ("AMD", vram)
    if "intel" in s_low:
        return ("Intel", vram)
    if "apple" in s_low or "m1" in s_low or "m2" in s_low:
        return ("Apple", vram)
    return ("Other", vram)

def simplify_os(x):
    if pd.isna(x): return "Other"
    s = str(x).lower()
    if "windows" in s:
        return "Windows"
    if "mac" in s or "apple" in s:
        return "Mac"
    if "linux" in s:
        return "Linux"
    return "Other"

# Parse CPU cores/threads (from the 'CPU' column which contains "Octa Core, 16 Threads" etc.)
word_to_number = {
    'octa': 8, 'hexa': 6, 'quad': 4, 'deca': 10, 'duo': 2, 'tri': 3, 'penta': 5
}
def parse_cores_threads(s):
    if pd.isna(s): return (np.nan, np.nan)
    s = str(s).lower()
    # try to find numeric cores
    cores = None
    threads = None
    m_core_num = re.search(r'(\d+)\s*cores?', s)
    if m_core_num:
        cores = int(m_core_num.group(1))
    else:
        # check words
        for w, n in word_to_number.items():
            if w in s:
                cores = n
                break
    m_threads = re.search(r'(\d+)\s*threads?', s)
    if m_threads:
        threads = int(m_threads.group(1))
    return (cores, threads)

# Parse processor column for brand, family, generation
def parse_processor(proc):
    if pd.isna(proc): return (np.nan, np.nan, np.nan)
    s = str(proc)
    s_low = s.lower()
    # brand
    if "intel" in s_low:
        brand = "Intel"
    elif "amd" in s_low:
        brand = "AMD"
    elif "apple" in s_low:
        brand = "Apple"
    else:
        brand = "Other"
    # family (i3/i5/i7/i9, ryzen 3/5/7/9)
    family = None
    fam_m = re.search(r'\b(i[3579])\b', s_low)  # intel i3,i5,i7,i9
    if fam_m:
        family = fam_m.group(1).upper()
    else:
        fam_m = re.search(r'\b(ryzen\s*[3579])\b', s_low)
        if fam_m:
            family = fam_m.group(1).title().replace(" ", "")
    # generation: look for "13th Gen" or patterns like leading digits in model '13900HX' -> 13
    gen = None
    m = re.search(r'(\d+)(?:th|rd|st|nd)\s+gen', s_low)
    if m:
        gen = int(m.group(1))
    else:
        # intel model like 13900HX -> take first two digits if they look like generation
        m2 = re.search(r'\b(i[3579])\s*[\w\s]*\s*(\d{2,4})', s_low)
        if m2:
            model_num = m2.group(2)
            # intel new gen: for 13900 -> 13
            if len(model_num) >= 3:
                gen = int(model_num[:2]) if int(model_num[:2]) <= 20 else int(model_num[0])
        else:
            # AMD Ryzen often: 5800H -> generation often first digit is series (5)
            m3 = re.search(r'ryzen\s*?(\d)', s_low)
            if m3:
                gen = int(m3.group(1))
    return (brand, family, gen)

# --- CLEANING PIPELINE ---
def clean_laptop_df(df):
    df = df.copy()
    # drop saved index columns & name (usually too unique)
    for col in ["Unnamed: 0.1", "Unnamed: 0", "name"]:
        if col in df.columns:
            df.drop(columns=col, inplace=True)
    # standardize brand capitalization
    if "brand" in df.columns:
        df["brand"] = df["brand"].astype(str).str.title()

    # RAM -> numeric GB
    if "Ram" in df.columns:
        df["Ram_GB"] = df["Ram"].apply(parse_ram).astype("Int64")

    # ROM -> numeric GB
    if "ROM" in df.columns:
        df["ROM_GB"] = df["ROM"].apply(to_gb_storage).astype("Float64")

    # Processor parsing (keep detailed original)
    if "processor" in df.columns:
        parsed = df["processor"].apply(parse_processor)
        df[["cpu_brand", "cpu_family", "cpu_generation"]] = pd.DataFrame(parsed.tolist(), index=df.index)
    # CPU column -> cores, threads
    if "CPU" in df.columns:
        parsed_ct = df["CPU"].apply(parse_cores_threads)
        df[["cpu_cores", "cpu_threads"]] = pd.DataFrame(parsed_ct.tolist(), index=df.index)
        # cast to Int64
        df["cpu_cores"] = df["cpu_cores"].astype("Int64")
        df["cpu_threads"] = df["cpu_threads"].astype("Int64")

    # GPU -> brand and VRAM (if present); keep original GPU if you want
    if "GPU" in df.columns:
        gp = df["GPU"].apply(gpu_brand_and_vram)
        df[["gpu_brand", "gpu_vram_gb"]] = pd.DataFrame(gp.tolist(), index=df.index)
        df["gpu_vram_gb"] = df["gpu_vram_gb"].astype("Float64")

    # OS simplify
    if "OS" in df.columns:
        df["OS_simple"] = df["OS"].apply(simplify_os)

    # display_size already numeric in sample; ensure numeric
    if "display_size" in df.columns:
        df["display_size"] = pd.to_numeric(df["display_size"], errors="coerce")

    # resolution columns ensure int
    for col in ["resolution_width", "resolution_height"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # warranty ensure int
    if "warranty" in df.columns:
        df["warranty"] = pd.to_numeric(df["warranty"], errors="coerce").astype("Int64")

    # Optionally drop old raw columns (uncomment if you want to remove originals)
    # drop_cols = ["Ram", "ROM", "GPU", "CPU", "processor", "OS"]
    # for c in drop_cols:
    #     if c in df.columns:
    #         df.drop(columns=c, inplace=True)

    return df

# run cleaning
df_clean = clean_laptop_df(df)

# quick check
print("CLEANED DF PREVIEW:")
display(df_clean.head(6))
print("\nINFO:")
print(df_clean.info())


CLEANED DF PREVIEW:


,brand,price,spec_rating,processor,CPU,Ram,Ram_type,ROM,ROM_type,GPU,...,Ram_GB,ROM_GB,cpu_brand,cpu_family,cpu_generation,cpu_cores,cpu_threads,gpu_brand,gpu_vram_gb,OS_simple
0,Hp,49900,73.000000,5th Gen AMD Ryzen 5 5600H,"Hexa Core, 12 Threads",8,DDR4,512GB,SSD,4GB AMD Radeon RX 6500M,...,8,512.0,AMD,Ryzen5,5.0,6,12,AMD,4.0,Windows
1,Hp,39900,60.000000,12th Gen Intel Core i3 1215U,"Hexa Core (2P + 4E), 8 Threads",8,DDR4,512GB,SSD,Intel UHD Graphics,...,8,512.0,Intel,I3,12.0,6,8,Intel,<NA>,Windows
2,Acer,26990,69.323529,11th Gen Intel Core i3 1115G4,"Dual Core, 4 Threads",8,DDR4,512GB,SSD,Intel Iris Xe Graphics,...,8,512.0,Intel,I3,11.0,<NA>,4,Intel,<NA>,Windows
3,Lenovo,59729,66.000000,12th Gen Intel Core i5 1240P,"12 Cores (4P + 8E), 16 Threads",16,LPDDR5,512GB,SSD,Intel Integrated Iris Xe,...,16,512.0,Intel,I5,12.0,12,16,Intel,<NA>,Windows
4,Apple,69990,69.323529,Apple M1,Octa Core (4P + 4E),8,DDR4,256GB,SSD,Apple M1 Integrated Graphics,...,8,256.0,Apple,None,NaN,8,<NA>,Apple,<NA>,Mac
5,Acer,39990,62.000000,12th Gen Intel Core i5 1240P,"12 Cores (4P + 8E), 16 Threads",8,DDR4,512GB,SSD,Intel Iris Xe Graphics,...,8,512.0,Intel,I5,12.0,12,16,Intel,<NA>,Windows



INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893 entries, 0 to 892
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              893 non-null    object 
 1   price              893 non-null    int64  
 2   spec_rating        893 non-null    float64
 3   processor          893 non-null    object 
 4   CPU                893 non-null    object 
 5   Ram                893 non-null    int64  
 6   Ram_type           893 non-null    object 
 7   ROM                893 non-null    object 
 8   ROM_type           893 non-null    object 
 9   GPU                893 non-null    object 
 10  display_size       893 non-null    float64
 11  resolution_width   893 non-null    Int64  
 12  resolution_height  893 non-null    Int64  
 13  OS                 893 non-null    object 
 14  warranty           893 non-null    Int64  
 15  Ram_GB             893 non-null    Int64  
 16  ROM_GB             

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:

df_clean.to_csv("/content/drive/MyDrive/laptop_price_project/data/laptops_cleaned.csv", index=False)
